In [35]:
import time
import requests
from flask import Flask, request
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [36]:
TOKEN = "1673852036:AAESmCKM-4bXapg1ZUutdZ5ogvd-n1ZSHVs"
PORT = 5000

# Определяем функции:

In [20]:
def read_server_url(): # вычитываем url выданный для нашего сервиса.
  with open('nohup.out', 'r') as f:
    return f.read().split(': ')[-1].split('\n')[0]

In [21]:
def make_api_url(method): 
  return f"https://api.telegram.org/bot{TOKEN}/{method}"

In [22]:
def set_webhook(server_url):
  api_url = make_api_url('setWebhook')
  post_data = {
      "url": server_url,
      }
  x = requests.post(api_url, data = post_data, headers={'header': 'Content-Type: application/json'})
  return x.json()['ok']

In [23]:
def send_message(chat_id, text):
    method = "sendMessage"
    url = make_api_url(method)
    data = {"chat_id": chat_id, "text": text}
    requests.post(url, data=data)

In [24]:
def get_photo_path(photo_id, server_url):
  api_url = make_api_url('getFile')
  post_data = {
      'file_id': photo_id,
      "url": server_url,
      }
  x = requests.post(api_url, data = post_data, headers={'header': 'Content-Type: application/json'})
  return x.json()['result']['file_path']

In [51]:
def get_photo(photo_id, server_url):
  photo_path = get_photo_path(photo_id, server_url)
  photo_url = f'https://api.telegram.org/file/bot{TOKEN}/{photo_path}'
  return  Image.open(requests.get(photo_url, stream=True).raw)

In [26]:
def process_photo(photo):
  plt.subplots()
  plt.imshow(photo)
  return 'result'

Препроцессинг

In [10]:
!git clone -l -s git://github.com/ZackPashkin/ocr_pytorch.git
!ls

Cloning into 'ocr_pytorch'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 252 (delta 136), reused 160 (delta 55), pack-reused 0
Receiving objects: 100% (252/252), 174.32 KiB | 10.89 MiB/s, done.
Resolving deltas: 100% (136/136), done.
ocr_pytorch  sample_data


In [11]:
!wget https://storage.googleapis.com/comptech2021-ocr/synth_dataset.zip
!unzip -qq /content/synth_dataset.zip -d /content/

--2021-02-04 17:57:48--  https://storage.googleapis.com/comptech2021-ocr/synth_dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.2.112, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2284647670 (2.1G) [application/zip]
Saving to: ‘synth_dataset.zip’

synth_dataset.zip   100%[===================>]   2.13G  30.7MB/s    in 26s     

2021-02-04 17:58:14 (84.3 MB/s) - ‘synth_dataset.zip’ saved [2284647670/2284647670]



In [14]:
!pip install -U albumentations

     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 37.6MB 83kB/s 
     |████████████████████████████████| 952kB 57.8MB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [3]:
%%time
import sys
import glob
import os
import pandas as pd
sys.path.append("/content/ocr_pytorch/src")
from PIL import Image, ImageOps
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import numpy as np
from albumentations import Compose, Normalize, MedianBlur, GaussianBlur, \
    MotionBlur, GaussNoise, MultiplicativeNoise, Cutout, \
    CoarseDropout, GridDistortion, ElasticTransform, \
    RandomBrightness, RandomContrast, RandomBrightnessContrast, \
    ShiftScaleRotate, IAAPiecewiseAffine

synth_kovshov = pd.read_csv("synth_dataset/kovshov_synth/crops_1.csv" ,sep='|', encoding = 'cp1251')

synth_labels_dict = {}
images_with_value = {}
for index, row in synth_kovshov.iterrows():
    split_row = row['0.png;(1839-1843), реформа'].split(";")
    img_path = "synth_dataset/kovshov_synth/crops_1/" + split_row[0]
    images_with_value.update({img_path:split_row[1]})

for index,image_name in enumerate(glob.glob("synth_dataset/kovshov_synth/crops_1/*.png")):
  for key in images_with_value:
    if (image_name == key):
      synth_labels_dict.update({key:images_with_value[key]})

synth_werner_list = []
synth_werner = open('synth_dataset/werner_distorted/annotation.txt','r')
try:
   synth_werner_list = synth_werner.read().splitlines()
finally:
   synth_werner.close()

images_with_value = {}
for index in range(len(synth_werner_list)):
  img_path = "synth_dataset/werner_distorted/content/generated/" + synth_werner_list[index].rpartition(',')[2] + ".png"
  label = synth_werner_list[index].rpartition(',')[0]
  images_with_value.update({img_path:label})

for index,image_name in enumerate(glob.glob("synth_dataset/werner_distorted/content/generated/*.png")):
  for key in images_with_value:
    if (image_name == key):
      synth_labels_dict.update({key:images_with_value[key]})

print(len(synth_labels_dict))

32291
CPU times: user 1min 3s, sys: 41.6 ms, total: 1min 3s
Wall time: 1min 3s


In [4]:
%%time

synth_werner_list = []
synth_werner = open('synth_dataset/werner_synthetic/annotation.txt','r')
try:
   synth_werner_list = synth_werner.read().splitlines()
finally:
   synth_werner.close()

for index in range(len(synth_werner_list)):
  img_path = "synth_dataset/werner_synthetic/content/result/" + synth_werner_list[index].partition(',')[0]
  label = synth_werner_list[index].partition(',')[2]
  synth_labels_dict.update({img_path:label})

print(len(synth_labels_dict))

272207
CPU times: user 238 ms, sys: 44 ms, total: 282 ms
Wall time: 286 ms


In [7]:
from sklearn import preprocessing
labels_dict = synth_labels_dict
IMAGES = list(labels_dict.keys())
# to look like '6bnnm'
LABELS_NAMES = list(labels_dict.values())
# to look like ['g', 'p', 'x', 'n', 'g']
LABELS_NAMES = [[_ for _ in x] for x in LABELS_NAMES]
print(LABELS_NAMES)
LABELS_NAMES_FLAT = [_ for sublist in LABELS_NAMES for _ in sublist]
print(LABELS_NAMES_FLAT)
labels_encoded = preprocessing.LabelEncoder()
labels_encoded.fit(LABELS_NAMES_FLAT)
print(labels_encoded.classes_)
print(len(labels_encoded.classes_))
# keep 0 for unknown
LABELS_ENCODED = np.array([labels_encoded.transform(x) for x in LABELS_NAMES]) +1
print(LABELS_ENCODED)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[' ' '!' '"' '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'
 ':' ';' '?' 'I' ']' '«' '»' '́' 'А' 'Б' 'В' 'Г' 'Д' 'Е' 'Ж' 'З' 'И' 'К'
 'Л' 'М' 'Н' 'О' 'П' 'Р' 'С' 'Т' 'У' 'Ф' 'Х' 'Ц' 'Ч' 'Ш' 'Э' 'Ю' 'Я' 'а'
 'б' 'в' 'г' 'д' 'е' 'ж' 'з' 'и' 'й' 'к' 'л' 'м' 'н' 'о' 'п' 'р' 'с' 'т'
 'у' 'ф' 'х' 'ц' 'ч' 'ш' 'щ' 'ъ' 'ы' 'ь' 'э' 'ю' 'я' 'ё' '–' '—' '“' '„']
90


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


[array([31, 68, 55, 70, 81, 63,  1, 58, 59, 67, 82,  8,  1, 40, 67])
 array([77, 59, 70, 67, 81, 59,  1, 55, 70, 68, 56, 62,  6,  1, 62])
 array([55, 68, 65, 82, 78, 68, 63,  1, 56, 59, 71, 72, 62, 55, 84, 65, 82,
        6,  1, 71])
 ... array([58, 59, 65, 62, 66, 68, 71, 72, 82])
 array([58, 59, 65, 62, 66, 68, 71, 72, 82])
 array([58, 59, 65, 62, 66, 68, 71, 72, 82])]


In [8]:


%matplotlib inline


def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [9]:
def transform_timeseries_string(batch):
    new_batch = []
    for string in batch:
        new_string = []
        last_symbol = '*'
        for symbol in string:
            if symbol == '*':
                pred_symbol = ''
            elif symbol == last_symbol:
                pass
            else:
                new_string.append(symbol)
                last_symbol = symbol
        new_batch.append(new_string)
    return new_batch

Объявление модели

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import models

class ResNetOCR(nn.Module):
  def __init__(self, n_classes, hidden_rnn):
    super(ResNetOCR,self).__init__()
    model = nn.Sequential(*list(models.resnet18(pretrained=False).children())[:-4])
    self.fc1 = nn.Linear(1024, hidden_rnn)
    self.cnn = model
    self.rnn = nn.GRU(hidden_rnn, 32 , bidirectional=True,
                          num_layers=2,
                          batch_first=True)
    self.output = nn.Linear(64,n_classes+1)
  
  def forward(self, images, labels=None, len_labels = None):
    bs, c, h, w = images.size()
    x = self.cnn(images)
    #print(x.size())
    x = x.permute(0,3,1,2)
    #print(x.size())
    x = x.view(bs, x.size(1),-1)
    #print(x.size()[2])
    x = self.fc1(x)
    x, _ = self.rnn(x)
    x = self.output(x)
    #print(x.size())
    # permute again 
    x = x.permute(1,0,2)
    #print(x)
    if labels is not None:
      log_softmax_values =  F.log_softmax(x,2)   
      input_lenghts = torch.full(size=(bs,),
                                 fill_value=log_softmax_values.size(0), 
                                 dtype = torch.int32
                                 )
      #print(input_lenghts)
         
      loss = nn.CTCLoss(blank=0,zero_infinity = True)(
          log_softmax_values,
          labels,
          input_lenghts,
          len_labels
      )  
      
      return x, loss
    return x

In [12]:
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 64
DEVICE = 'cuda'


In [13]:
model_train = ResNetOCR(n_classes=len(labels_encoded.classes_),hidden_rnn=64)
model_train.load_state_dict(torch.load("rn18.pt"))
model_train.to(DEVICE)
model_train.eval() # переключаем нейросеть в режим обучения

ResNetOCR(
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps

Модель

In [14]:
from decode_predictions import decode_preds



In [50]:
def pred(images):
  augmentations = Compose([Normalize(always_apply=True),])

  list_tensors = []
  for index, image in enumerate(images):
    image = image.convert('RGB')
    #image = Image.fromarray(image)

    image = resize_with_padding(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT), resample=Image.BILINEAR)   
    #display(image)
    image = np.array(image)
    augmented_image = augmentations(image=image)
    image = augmented_image["image"]
    image = np.transpose(image,(2,0,1)).astype(np.float32)
    image = torch.tensor(image, dtype=torch.float)
    list_tensors.append(image)
    #if index == 0: break
    #image = image[None, :, :, :]

  res = torch.stack(list_tensors)

  batch_prediction = model_train(res.to(DEVICE))
  cur_preds = decode_preds(batch_prediction, labels_encoded)
  pred_label_list = []
  valid_final_preds = transform_timeseries_string(cur_preds)
  for i in range(len(valid_final_preds)):
    pred_label = ''.join(map(str, valid_final_preds[i]))
    pred_label_list.append(pred_label)

  return pred_label_list

  

# Запускаем сервер в гугл колабе, который отвечает за то, чтобы присвоить нашему выделенному серверу в гугл колабе определённый url.

In [58]:
#in terminal
!curl -sL https://deb.nodesource.com/setup_14.x -o nodesource_setup.sh
!bash nodesource_setup.sh
!apt install nodejs

!npm install -g localtunnel
! nohup lt --port {PORT} &
time.sleep(1) 
# take url like https://wicked-skunk-16.loca.lt


## Installing the NodeSource Node.js 14.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 https://deb.nodesource.com/node_14.x bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bi

In [59]:
server_url = read_server_url() # вычитываем выданный нам url

In [60]:
server_url

'https://cuddly-rattlesnake-92.loca.lt'

In [61]:
set_webhook(server_url) # создаём хук, чтобы телеграм отправлял нам уведомления о новых сообщениях

True

In [62]:
# Выводим текст, если пользователь не отправил фото.

TEXT_IMAGE_EMPTY = """Этот бот предназначен для обработки изображения и распознавания текста. Отправьте, пожалуйста, фото, текст на котором вам необходимо распознать.
"""
TEXT_IMAGE_START = """Привет, я умею распознавать рукописный текст. Отправь мне фото текста, который нужно распознать."""

In [ ]:
#Запускаем фласк. 

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def receive_update():
    if request.method == "POST":
        print(request.json)
        chat_id = request.json["message"]["chat"]["id"]
        photos = request.json["message"].get("photo")
        print("entities" in request.json["message"])
        downloaded_photo_list = []
        if photos:
            photo = photos[-1]
            
            downloaded_photo = get_photo(photo['file_id'], server_url)
            downloaded_photo_list.append(downloaded_photo)
            result = pred(downloaded_photo_list)
            send_message(chat_id, *result)
            print(*result)
            
        elif "entities" in request.json["message"]:
          if request.json["message"]['entities'][0]['type'] == 'bot_command':
            if request.json["message"]["text"] == "/start":
              send_message(chat_id, TEXT_IMAGE_START)
        else:
          send_message(chat_id, TEXT_IMAGE_EMPTY)
    return 'result'

app.run(port=PORT)

#attach image in telegram

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'update_id': 294559937, 'message': {'message_id': 250, 'from': {'id': 209041395, 'is_bot': False, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'language_code': 'ru'}, 'chat': {'id': 209041395, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'type': 'private'}, 'date': 1612462406, 'photo': [{'file_id': 'AgACAgIAAxkBAAP6YBw5RkrnptWZlR6n19Q_vin4sVsAAoOxMRtbf-FIgun6OvjDgqhNzQ2eLgADAQADAgADbQADd2MBAAEeBA', 'file_unique_id': 'AQADTc0Nni4AA3djAQAB', 'file_size': 4404, 'width': 320, 'height': 48}, {'file_id': 'AgACAgIAAxkBAAP6YBw5RkrnptWZlR6n19Q_vin4sVsAAoOxMRtbf-FIgun6OvjDgqhNzQ2eLgADAQADAgADeAADdWMBAAEeBA', 'file_unique_id': 'AQADTc0Nni4AA3VjAQAB', 'file_size': 4914, 'width': 398, 'height': 60}]}}
False


127.0.0.1 - - [04/Feb/2021 18:27:22] "POST / HTTP/1.1" 200 -


{'update_id': 294559938, 'message': {'message_id': 251, 'from': {'id': 209041395, 'is_bot': False, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'language_code': 'ru'}, 'chat': {'id': 209041395, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'type': 'private'}, 'date': 1612462692, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}
True


...

127.0.0.1 - - [04/Feb/2021 18:42:07] "POST / HTTP/1.1" 200 -


{'update_id': 294559964, 'message': {'message_id': 301, 'from': {'id': 209041395, 'is_bot': False, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'language_code': 'ru'}, 'chat': {'id': 209041395, 'first_name': 'Александр', 'last_name': 'Тараканов', 'username': 'sunndoz', 'type': 'private'}, 'date': 1612464159, 'photo': [{'file_id': 'AgACAgIAAxkBAAIBLWAcQB-sLoIjigJ4kzoN2XOcKmHKAAKWsTEbW3_hSNz1rfXXc78sveJNni4AAwEAAwIAA20AA6xwAQABHgQ', 'file_unique_id': 'AQADveJNni4AA6xwAQAB', 'file_size': 3435, 'width': 320, 'height': 67}, {'file_id': 'AgACAgIAAxkBAAIBLWAcQB-sLoIjigJ4kzoN2XOcKmHKAAKWsTEbW3_hSNz1rfXXc78sveJNni4AAwEAAwIAA3kAA6lwAQABHgQ', 'file_unique_id': 'AQADveJNni4AA6lwAQAB', 'file_size': 11361, 'width': 936, 'height': 196}, {'file_id': 'AgACAgIAAxkBAAIBLWAcQB-sLoIjigJ4kzoN2XOcKmHKAAKWsTEbW3_hSNz1rfXXc78sveJNni4AAwEAAwIAA3gAA6pwAQABHgQ', 'file_unique_id': 'AQADveJNni4AA6pwAQAB', 'file_size': 12568, 'width': 800, 'height': 168}]}}
False


127.0.0.1 - - [04/Feb/2021 18:42:40] "POST / HTTP/1.1" 200 -


{'update_id': 294559965, 'message': {'message_id': 303, 'from': {'id': 289630251, 'is_bot': False, 'first_name': 'Денис', 'username': 'Sovenq', 'language_code': 'ru'}, 'chat': {'id': 289630251, 'first_name': 'Денис', 'username': 'Sovenq', 'type': 'private'}, 'date': 1612464217, 'photo': [{'file_id': 'AgACAgIAAxkBAAIBL2AcQFnssUTSE60SJSKng_mhaKdGAALVsjEbg-bhSOXVmH9t4FsTmC5EmC4AAwEAAwIAA20AA5uTBgABHgQ', 'file_unique_id': 'AQADmC5EmC4AA5uTBgAB', 'file_size': 2281, 'width': 164, 'height': 148}]}}
False


127.0.0.1 - - [04/Feb/2021 18:43:38] "POST / HTTP/1.1" 200 -
